In [1]:
!apt-get update
!apt-get install memcached
!pip install pymemcache

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,563 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,631 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,840 k

This command starts Memcached as the nobody user on port 11211.

In [2]:
!memcached -d -p 11211 -u nobody


In [3]:
import requests as rq
import json
import pandas as pd
from pandas import json_normalize
import pickle  # For caching data
import os  # To manage the process ID
import psutil  # For monitoring CPU and memory usage
import time  # For execution time measurement
from pymemcache.client import base


**ADAPTER**

In [4]:
import requests as rq
import pandas as pd
import pickle
from pymemcache.client import base

class EnergyAdapter1:
    def __init__(self):
        self.base_url = 'https://odre.opendatasoft.com/api/records/1.0/search/?dataset=consommation-quotidienne-brute-regionale&q=&sort=-consommation_brute_gaz_grtgaz&facet=date_heure&facet=code_insee_region&facet=region'
        self.list_of_cols = ['region', 'code_insee_region', 'date_heure', 'date', 'heure', 'consommation_brute_electricite_rte', 'consommation_brute_totale', 'consommation_brute_gaz_totale']

    def get_energy_production(self):
        try:
            response = rq.get(self.base_url)
            response.raise_for_status()  # Ensure we catch HTTP errors
            parse_json = response.json()  # Parse the JSON response
            print(f"EnergyAdapter1 raw response: {parse_json}")  # Debug: print raw API response
            records = parse_json.get('records', [])
            data = pd.DataFrame([x['fields'] for x in records])
            data = data[self.list_of_cols]
            return data
        except Exception as e:
            print(f"Error fetching data from API 1: {e}")
            return pd.DataFrame()  # Return empty DataFrame on error
class EnergyAdapter2:
    def __init__(self):
        self.base_url = "https://opendata.agenceore.fr/api/records/1.0/search/?dataset=production-demi-horaire-agregee-par-region&q=&sort=-horodate&facet=horodate&facet=region&facet=grd"
        self.list_of_cols = ['region', 'horodate', 'energie_injectee', 'nb_points_injection']

    def get_energy_production(self):
        try:
            response = rq.get(self.base_url)
            response.raise_for_status()
            parse_json = response.json()
            print(f"EnergyAdapter2 raw response: {parse_json}")  # Debug: print raw API response
            records = parse_json.get('records', [])
            data = pd.DataFrame([x['fields'] for x in records])
            data = data[self.list_of_cols]
            return data
        except Exception as e:
            print(f"Error fetching data from API 2: {e}")
            return pd.DataFrame()  # Return empty DataFrame on error
class PopulationAdapter:
    def __init__(self):
        self.base_url = "https://public.opendatasoft.com/api/records/1.0/search/?dataset=demographyref-france-pop-legale-commune-arrondissement-municipal-millesime&q=&rows=3536&facet=reg_code&facet=reg_name&facet=com_arm_code&facet=com_arm_name&facet=dep_code&facet=arrdep_code&facet=census_year&facet=start_year&facet=geo_year&facet=epci_name&facet=epci_code&facet=dep_name"
        self.list_of_cols = ['com_arm_pop_tot', 'reg_name', 'start_year', 'arrdep_code', 'geo_year', 'census_year', 'com_arm_code', 'com_arm_pop_mun', 'dep_code', 'com_arm_pop_cap', 'reg_code', 'com_arm_name']

    def get_population(self):
        try:
            response = rq.get(self.base_url)
            response.raise_for_status()
            parse_json = response.json()
            print(f"PopulationAdapter raw response: {parse_json}")  # Debug: print raw API response
            records = parse_json.get('records', [])
            data = pd.DataFrame([x['fields'] for x in records])
            data = data[self.list_of_cols]
            data = data.rename(columns={"reg_name": "region"})
            return data
        except Exception as e:
            print(f"Error fetching data from Population API: {e}")
            return pd.DataFrame()  # Return empty DataFrame on error
class LogementAdapter:
    def __init__(self):
        self.base_url = "https://opendata.caissedesdepots.fr/api/records/1.0/search/?dataset=bailleurs_sociaux_region&q=&rows=1000&facet=code_region&facet=libelle_region&facet=annee"
        self.list_of_cols = ['part_logement_sociaux_geres_sem', 'nbre_logements', 'code_region', 'nbre_bailleurs_consolide', 'part_logements_sociaux_geres_autres_bailleurs', 'nbre_bailleurs', 'annee', 'libelle_region', 'part_logement_sociaux_geres_esh', 'part_logement_sociaux_geres_oph']

    def API_Logement_sociaux_bailleurs(self):
        try:
            response = rq.get(self.base_url)
            response.raise_for_status()
            parse_json = response.json()
            print(f"LogementAdapter raw response: {parse_json}")  # Debug: print raw API response
            records = parse_json.get('records', [])
            data = pd.DataFrame([x['fields'] for x in records])
            data = data[self.list_of_cols]
            data = data.rename(columns={"libelle_region": "region"})
            return data
        except Exception as e:
            print(f"Error fetching data from Logement API: {e}")
            return pd.DataFrame()  # Return empty DataFrame on error




In [15]:
# The InformationMediator class remains the same, only fetching data from real APIs now.
class InformationMediator:
    def __init__(self):
        self.energy_adapter1 = EnergyAdapter1()
        self.energy_adapter2 = EnergyAdapter2()
        self.population_adapter = PopulationAdapter()
        self.logement_adapter = LogementAdapter()
        self.cache = base.Client(('localhost', 11211))  # Connect to Memcached

    def get_cache_key(self, query):
        """Generate a unique cache key based on the query."""
        return f"cache_{hash(query)}"

    def cache_data(self, query, data, expire=300):
      """Store data in cache with an expiration time (in seconds)."""
      cache_key = self.get_cache_key(query)
      # Encode DataFrame as a binary-safe pickle
      self.cache.set(cache_key, pickle.dumps(data), expire=expire)

    def get_cached_data(self, query):
      """Retrieve data from cache."""
      cache_key = self.get_cache_key(query)
      cached_data = self.cache.get(cache_key)
      if cached_data is not None:
          return pickle.loads(cached_data)  # Decode the cached pickle data
      return None

    def get_API_data(self, query, api_function):
        """Check cache before querying the API and ensure the result is a valid DataFrame."""
        cached_data = self.get_cached_data(query)
        if cached_data is not None:
            return cached_data

        data = api_function()  # Fetch data from the API

        if not isinstance(data, pd.DataFrame):
            raise ValueError("The fetched data is not a valid DataFrame")

        if data.empty:
            raise ValueError("The fetched DataFrame is empty")

        # Cache the result
        self.cache_data(query, data)

        return data

    def get_API1_data(self, query):
        return self.get_API_data(query, self.energy_adapter1.get_energy_production)

    def get_API2_data(self, query):
        return self.get_API_data(query, self.energy_adapter2.get_energy_production)

    def get_API3_data(self, query):
        return self.get_API_data(query, self.population_adapter.get_population)

    def get_API4_data(self, query):
        return self.get_API_data(query, self.logement_adapter.API_Logement_sociaux_bailleurs)

    def data(self, query):
      """Fetch data from cache or APIs and merge them."""
      cached_data = self.get_cached_data(query)
      if cached_data:
          return cached_data

      try:
          # Fetch data from APIs
          df1 = self.get_API1_data(query)
          df2 = self.get_API2_data(query)
          df3 = self.get_API3_data(query)
          df4 = self.get_API4_data(query)

          # Ensure only existing columns are selected
          df1 = df1[['region', 'consommation_brute_electricite_rte', 'date', 'heure']] if not df1.empty else pd.DataFrame()
          df2 = df2[['region', 'energie_injectee']] if 'energie_injectee' in df2.columns else pd.DataFrame()
          df3 = df3[['region', 'com_arm_pop_tot']] if 'com_arm_pop_tot' in df3.columns else pd.DataFrame()
          df4 = df4[['region', 'part_logement_sociaux_geres_sem']] if 'part_logement_sociaux_geres_sem' in df4.columns else pd.DataFrame()

      except Exception as e:
          print(f"Error while fetching data: {e}")
          return None

      # Perform merges
      try:
          df = pd.merge(df1, df2, on="region", how='inner') if not df2.empty else df1
          df = pd.merge(df, df3, on="region", how='inner') if not df3.empty else df
          df = pd.merge(df, df4, on="region", how='inner') if not df4.empty else df
      except Exception as e:
          print(f"Error occurred during merging: {e}")
          return None

      self.cache_data(query, df)
      return df


In [16]:
# Define a sample query
FirstQuery = "SELECT region, heure, consommation_brute_electricite_rte, energie_injectee FROM df"

# Create an instance of InformationMediator
mediator = InformationMediator()

# Clear all cache to avoid corrupted data issues
mediator.cache.flush_all()  # Flushing the Memcached cache

# Proceed with running the query
pid = os.getpid()
py = psutil.Process(pid)

cpuStart = py.cpu_percent()
memoryStart = py.memory_info()[0] / 2. ** 30
start_time = time.time()

# Run the query using InformationMediator
result = mediator.data(FirstQuery)
print(result)

elapsed_time = abs(time.time() - start_time)
cpuEnd = py.cpu_percent()
memoryEnd = py.memory_info()[0] / 2. ** 30

cpu_usage = abs(cpuEnd - cpuStart)
memory_usage = abs(memoryEnd - memoryStart)

print("Execution Time:", elapsed_time, "seconds.")
print("CPU Usage:", cpu_usage, "percent.")
print("Memory Usage:", memory_usage, "GB.")


EnergyAdapter1 raw response: {'nhits': 2383476, 'parameters': {'dataset': 'consommation-quotidienne-brute-regionale', 'rows': 10, 'start': 0, 'sort': ['-consommation_brute_gaz_grtgaz'], 'facet': ['date_heure', 'code_insee_region', 'region'], 'format': 'json', 'timezone': 'UTC'}, 'records': [{'datasetid': 'consommation-quotidienne-brute-regionale', 'recordid': 'a6938d7f1bf2a63c96b0ad099b521f64f5d79c59', 'fields': {'date': '2022-03-26', 'heure': '02:00', 'consommation_brute_electricite_rte': 4686, 'region': 'Grand Est', 'date_heure': '2022-03-26T01:00:00+00:00', 'statut_grtgaz': 'Redressé', 'consommation_brute_gaz_grtgaz': 0, 'statut_rte': 'Consolidé', 'consommation_brute_gaz_totale': 0, 'code_insee_region': '44', 'consommation_brute_totale': 4686}, 'record_timestamp': '2023-12-07T19:57:52.882Z'}, {'datasetid': 'consommation-quotidienne-brute-regionale', 'recordid': 'e21c3923b9e2cf959876f54575916e3a9c1ed18c', 'fields': {'date': '2023-03-26', 'heure': '02:00', 'consommation_brute_electric